# Bootes Brown photo-z cross

The photometric redshifts are calculated for the Brown aperture matched catalogue. Since in the manufacture of the masterlist we take the Brown aperture matched catalogue and add in some other obejcts. We here asign the Brown objects with their help id

## 1. Load catalogues and join on IRAC ID

In [1]:
from astropy.table import Table, join, Column
import numpy as np

In [2]:
photo_z = Table.read('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public.fits')[
    'ID',
    'ID_OPTICAL', 
    'z1_median'
]
cross_ids = Table.read('../../dmu1/dmu1_ml_Bootes/data/'
                     + 'master_list_cross_ident_bootes_20190701.fits')[
    'help_id', 
    'brown_i_id'
]
cross_ids['brown_i_id'].name = 'ID'


photo_z['ID'].fill_value = -99
photo_z = photo_z.filled()

In [3]:
new_cat = join(photo_z, cross_ids, join_type='left')

## 2. Replace bad photo-zs with nans

In [4]:
new_cat[new_cat['z1_median']<0]['z1_median'] = np.nan

In [5]:
new_cat['help_id'].fill_value = ''
new_cat = new_cat.filled()

## 3. Remove objects with duplicate help_ids

There seem to be a number of objects with duplicate ids which becaome duplicate help_ids. As far as I can tell these are totally identical so I remove the duplicates.

In [6]:
np.sum(new_cat['help_id'] == '')

556961

In [7]:
len(new_cat)

2214845

In [8]:
new_cat = new_cat[~(new_cat['help_id'] == '')]

In [9]:
len(new_cat)

1657884

In [10]:
new_cat['help_id'].dtype

dtype('S27')

In [11]:
len(np.unique(new_cat['help_id']))

1657470

In [12]:
new_cat.sort('help_id')
new_cat['help_id'][:-1][new_cat['help_id'][1:] == new_cat['help_id'][:-1]]

HELP_J142437.818+334207.903
HELP_J142439.779+341546.007
HELP_J142441.422+334445.487
HELP_J142442.081+322823.807
HELP_J142451.435+351158.274
HELP_J142452.724+344704.613
HELP_J142454.282+341933.772
HELP_J142458.833+341759.494
HELP_J142459.227+330212.781
HELP_J142500.137+333627.582
HELP_J142500.994+341654.105


In [13]:
new_cat[new_cat['help_id']=='HELP_J142437.818+334207.903']

ID,ID_OPTICAL,z1_median,help_id
int64,int64,float64,bytes27
691549,691550,0.3343,HELP_J142437.818+334207.903
691549,691550,0.3343,HELP_J142437.818+334207.903


In [14]:
is_duplicate = new_cat['help_id'][1:] == new_cat['help_id'][:-1]
#np.append(is_duplicate,False)
new_cat = new_cat[~np.append(is_duplicate,False)]

In [15]:
new_cat[new_cat['help_id']=='HELP_J142437.818+334207.903']

ID,ID_OPTICAL,z1_median,help_id
int64,int64,float64,bytes27
691549,691550,0.3343,HELP_J142437.818+334207.903


In [16]:
len(new_cat)

1657470

In [17]:
len(new_cat) == len(np.unique(new_cat['help_id']))

True

In [18]:
new_cat[:10].show_in_notebook()

idx,ID,ID_OPTICAL,z1_median,help_id
0,1788580,1788581,0.7797,HELP_J142433.597+352308.103
1,1891099,1891100,2.5117,HELP_J142433.602+353326.121
2,1997888,1997889,0.2227,HELP_J142433.609+354416.174
3,1776606,1776607,0.8996,HELP_J142433.611+352156.195
4,1780409,1780410,0.8052,HELP_J142433.614+352219.215
5,2003639,2003640,0.1278,HELP_J142433.614+354452.719
6,1729999,1730000,0.3376,HELP_J142433.625+351736.189
7,1888110,1888111,0.5268,HELP_J142433.629+353308.009
8,2000812,2000813,0.1662,HELP_J142433.635+354435.055
9,2029549,2029550,1.9393,HELP_J142433.641+354736.024


## 4. Write catalogue
This will be the catalogues that is used in dmu32 to merge photozs

In [19]:
new_cat.write('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public_with_help_id.fits', overwrite=True)